In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from exnn import ExNN

def data_generator1(datanum, testnum=10000, noise_sigma=1, rand_seed=0):
    
    corr = 0.5
    np.random.seed(rand_seed)
    proj_matrix = np.zeros((10, 4))
    proj_matrix[:7, 0] = np.array([1,0,0,0,0,0,0])
    proj_matrix[:7, 1] = np.array([0,1,0,0,0,0,0])
    proj_matrix[:7, 2] = np.array([0,0,0.5,0.5,0,0,0])
    proj_matrix[:7, 3] = np.array([0,0,0,0,0.2,0.3,0.5])
    u = np.random.uniform(-1, 1, [datanum + testnum, 1])
    t = np.sqrt(corr / (1 - corr))
    x = np.zeros((datanum + testnum, 10))
    for i in range(10):
        x[:, i:i + 1] = (np.random.uniform(-1, 1, [datanum + testnum, 1]) + t * u) / (1 + t)

    y = np.reshape(2 * np.dot(x, proj_matrix[:, 0]) + 0.2 * np.exp(-4 * np.dot(x, proj_matrix[:, 1])) + \
                   3 * (np.dot(x, proj_matrix[:, 2]))**2 + 2.5 * np.sin(np.pi * np.dot(x, proj_matrix[:, 3])), [-1, 1]) + \
              noise_sigma * np.random.normal(0, 1, [datanum + testnum, 1])
    
    task_type = "Regression"
    meta_info = {"X1":{"type":"continuous"},
             "X2":{"type":"continuous"},
             "X3":{"type":"continuous"},
             "X4":{"type":"continuous"},
             "X5":{"type":"continuous"},
             "X6":{"type":"continuous"},
             "X7":{"type":"continuous"},
             "X8":{"type":"continuous"},
             "X9":{"type":"continuous"},
             "X10":{"type":"continuous"},
             "Y":{"type":"target"}}
    for i, (key, item) in enumerate(meta_info.items()):
        if item['type'] == "target":
            sy = MinMaxScaler((-1, 1))
            y = sy.fit_transform(y)
            meta_info[key]["scaler"] = sy
        elif item['type'] == "categorical":
            enc = OrdinalEncoder()
            enc.fit(x[:,[i]])
            ordinal_feature = enc.transform(x[:,[i]])
            x[:,[i]] = ordinal_feature
            meta_info[key]["values"] = enc.categories_[0].tolist()
        else:
            sx = MinMaxScaler((-1, 1))
            x[:,[i]] = sx.fit_transform(x[:,[i]])
            meta_info[key]["scaler"] = sx

    train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=testnum, random_state=rand_seed)
    return train_x, test_x, train_y, test_y, task_type, meta_info

train_x, test_x, train_y, test_y, task_type, meta_info = data_generator1(datanum=10000, testnum=10000, noise_sigma=1, rand_seed=0)
model = ExNN(meta_info=meta_info,
               subnet_num=10,
               subnet_arch=[10, 6],
               task_type=task_type,
               activation_func=tf.tanh,
               batch_size=min(1000, int(train_x.shape[0] * 0.2)),
               training_epochs=5000,
               lr_bp=0.001,
               lr_cl=0.1,
               beta_threshold=0.05,
               tuning_epochs=100,
               l1_proj=0.0001,
               l1_subnet=0.00316,
               l2_smooth=10**(-6),
               verbose=True,
               val_ratio=0.2,
               early_stop_thres=100)

model.fit(train_x, train_y)

Initial training.
Training epoch: 1, train loss: 0.15248, val loss: 0.15325
Training epoch: 2, train loss: 0.13367, val loss: 0.13557
Training epoch: 3, train loss: 0.11582, val loss: 0.11764
Training epoch: 4, train loss: 0.09918, val loss: 0.10050
Training epoch: 5, train loss: 0.08543, val loss: 0.08640
Training epoch: 6, train loss: 0.07818, val loss: 0.07827
Training epoch: 7, train loss: 0.06875, val loss: 0.06865
Training epoch: 8, train loss: 0.06563, val loss: 0.06523
Training epoch: 9, train loss: 0.05757, val loss: 0.05775
Training epoch: 10, train loss: 0.05770, val loss: 0.05791
Training epoch: 11, train loss: 0.04955, val loss: 0.05008
Training epoch: 12, train loss: 0.04903, val loss: 0.04945
Training epoch: 13, train loss: 0.04844, val loss: 0.04874
Training epoch: 14, train loss: 0.04466, val loss: 0.04488
Training epoch: 15, train loss: 0.04277, val loss: 0.04296
Training epoch: 16, train loss: 0.03933, val loss: 0.03972
Training epoch: 17, train loss: 0.04030, val lo

Training epoch: 140, train loss: 0.02012, val loss: 0.02056
Training epoch: 141, train loss: 0.02019, val loss: 0.02076
Training epoch: 142, train loss: 0.01987, val loss: 0.02043
Training epoch: 143, train loss: 0.01997, val loss: 0.02046
Training epoch: 144, train loss: 0.01988, val loss: 0.02035
Training epoch: 145, train loss: 0.01988, val loss: 0.02046
Training epoch: 146, train loss: 0.01991, val loss: 0.02040
Training epoch: 147, train loss: 0.01995, val loss: 0.02058
Training epoch: 148, train loss: 0.02026, val loss: 0.02087
Training epoch: 149, train loss: 0.01972, val loss: 0.02024
Training epoch: 150, train loss: 0.02097, val loss: 0.02165
Training epoch: 151, train loss: 0.01993, val loss: 0.02054
Training epoch: 152, train loss: 0.02037, val loss: 0.02087
Training epoch: 153, train loss: 0.01970, val loss: 0.02024
Training epoch: 154, train loss: 0.02011, val loss: 0.02082
Training epoch: 155, train loss: 0.01970, val loss: 0.02024
Training epoch: 156, train loss: 0.01988

Training epoch: 277, train loss: 0.01799, val loss: 0.01862
Training epoch: 278, train loss: 0.01796, val loss: 0.01860
Training epoch: 279, train loss: 0.01776, val loss: 0.01848
Training epoch: 280, train loss: 0.01773, val loss: 0.01839
Training epoch: 281, train loss: 0.01813, val loss: 0.01881
Training epoch: 282, train loss: 0.01782, val loss: 0.01853
Training epoch: 283, train loss: 0.01800, val loss: 0.01867
Training epoch: 284, train loss: 0.01795, val loss: 0.01855
Training epoch: 285, train loss: 0.01767, val loss: 0.01831
Training epoch: 286, train loss: 0.01791, val loss: 0.01875
Training epoch: 287, train loss: 0.01810, val loss: 0.01886
Training epoch: 288, train loss: 0.01813, val loss: 0.01887
Training epoch: 289, train loss: 0.01770, val loss: 0.01835
Training epoch: 290, train loss: 0.01797, val loss: 0.01858
Training epoch: 291, train loss: 0.01809, val loss: 0.01890
Training epoch: 292, train loss: 0.01807, val loss: 0.01889
Training epoch: 293, train loss: 0.01761

In [ ]:
model.visualize(folder="./", name="exnn_demo")

In [ ]:
model.visualize_new(cols_per_row=3, subnet_num=3, dummy_subnet_num=0, folder="./", name="exnn_demo")